# Práctica Guiada: Scrapeando letras de canciones para niñes
### Germán Rosati (IDAES-UNSAM, CONICET, PIMSA)


## Introducción
---
La idea de este tutorial es avanzar en un ejemplo de scraping un poco más compejo. Vamos a hacer dos cosas:

1. Scrapear letras de canciones infantiles (sí, soy padre hace poco...)
2. Utilizar la API de Spotify para descargar algunos audio features


Primero, importamos los paquetes que vamos a usar:


In [0]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

Lo primero que tenemos que hacer es hacer un request al sitio que queremos scrapear, extraer su contenido y "parsearlo" para obtener la estructura del árbol de html:

In [0]:
links_me = requests.get('https://www.musica.com/letras.asp?letras=37155')
links_me = links_me.content
soup_me = BeautifulSoup(links_me)

Veamos qué es lo que devuelve el objeto `soup_me` 

In [0]:
soup_me.head

Puede verse que lo que obtenemos es una representación (más o menos) clara del árbol html del sitio. 

Si nos fijamos bien, podemos ver que las letras de M. E. Walsh aparecen como un link dentro del sitio. De hecho, aparecen dentro de una clase: 
```{html}
<div class="letra_data"><a href="letras.asp?letra=1852580">El reino del reves</a><p>Maria Elena Walsh</p></div></li><li class=" li50" onclick="window.location='letras.asp?letra=1814885'">
```

Como vimos antes, podríamos ir ingresando a cada subdivisión del árbol, pero el código se haría un tanto ilegible. 

Es por eso que vamos a usar una estrategia más "violenta", digamos: vamos a traer todos los links del sitio y nos vamos a quedar con los que nos interesen. Eso es lo que hace la siguiente función:

In [0]:
def getLinks(url):
  html = urlopen(url) # Abre la conexióón
  bsObj = BeautifulSoup(html) # Parsea el objeto
  links = []
  for l in bsObj.find_all("a", href=True): # Recorre todos los links
    if ('letra=' in l.attrs['href']) & ~(l.has_attr('title')): 
      links.append('https://www.musica.com/' + l.attrs['href'])
    else:
      pass
  return(links)

* La primera línea de la función, abre la conexión con el sitio que queremos scrapear
* La segunda parsea el código html del sitio y le da una estructura más legible para nuestros ojos
* El `for` loop itera sobre cada uno de los elementos con links y testea dos condiciones

Generamos, entonces, una lista con cada uno de los links de las letras de M. E. Walsh en el sitio.

In [0]:
links = getLinks('https://www.musica.com/letras.asp?letras=37155')
links

Ahora, sí podemos pasar a extraer las letras.

## Extrayendo letras de un sitio
---

Vamos por partes. En principio, cada link nos lleva a un sitio en el que deberia haber una letra de un tema de M. E. Walsh. Ahora bien, vamos a testear un poco el asunto.

Vayamos al link número 10 de nuestra lista de links y hagamos un testeo rápido. Nos tocó "Canción para tomar el té".

In [0]:
links[7]

'https://www.musica.com/letras.asp?letra=1818721'

In [0]:
links_me = requests.get(links[7])
links_me = links_me.content
soup_me = BeautifulSoup(links_me)

Ya parseamos el sitio, sería un buen momento para pensar qué información queremos extraer del sitio. En este caso, es relativamente simple: vamos a conservar el título y la letra.



Si miramos un poco el código de la página vamos a ver que los títulos están taggeados como `h3`. Entonces, es fácil traerlos:

In [0]:
soup_me.find('h3').get_text()

'Cancion  para bañar la luna (Letra/Lyrics)'

Las letras son un poco más complicadas. 

Lo que vamos a hacer es iterar sobre el objeto `soup_me` (que es el que tiene el sitio web de una letra) y nos vamos a meter en el atributo `<div><class:letra>`.

Veamos un poco el [html original](https://www.musica.com/letras.asp?letra=1818721)...

Está, efectivamente, un poco sucio, entonces


Pero como el html está un poco sucio tenemos que escribir un poco de código ad-hoc. Vamos a iterar sobre todos los objetos `div` que contengan los atributos `class` y `letra` y dentro de ellos los que tienen el tag `p`:

In [0]:
for i in soup_me.find('div', {'class':'letra'}).find_all('p'): 
  print(i)

<p><a href="https://www.musica.com/letras.asp?letras=37155">Maria Elena Walsh</a></p>
<p>Ya la Luna baja en camisón<br/>a bañarse en un charquito con jabón.<br/>Ya la Luna baja en tobogán<br/>revoleando su sombrilla de azafrán.<br/>Quien la pesque con una cañita de bambú,<br/>se la lleva a Siu Kiu.</p>
<p>Ya la luna viene en palanquin<br/>a robar un crisantemo del jardín<br/>Ya la luna viene por allí<br/>su kimono dice no, no y ella sí.<br/>Quien la pesque con una cañita de bambú,<br/>se la lleva a Siu Kiu.</p>
<p>Ya la luna baja muy feliz<br/>a empolvarse con azucar la nariz<br/>Ya la luna en puntas de pie<br/>en una tacita china toma té<br/>Quien la pesque con una cañita de bambú,<br/>se la lleva a Siu Kiu.</p>
<p>Ya la luna vino y le dio tos<br/>por comer con dos palitos el arroz<br/>Ya la luna baja desde allá<br/>y por el charquito-quito nadará<br/>Quien la pesque con una cañita de bambú,<br/>se la lleva a Siu Kiu</p>
<p class="hachedos">Datos de Cancion  para bañar la luna</p>
<p 

Vemos que hay unos cuantos que no queremos porque no son la letra propiamente dicha de la canción. Lo interesante es que son fáciles de filtrar: todos tienen el atributo `a` o bien la etiqueta `class`. Entonces, 


In [0]:
for i in soup_me.find('div', {'class':'letra'}).find_all('p'): 
  if i.has_attr('class'):
    break
  if i.a:
    pass
  else:
    print(i.get_text(strip=True, separator=' '))

Ya la Luna baja en camisón a bañarse en un charquito con jabón. Ya la Luna baja en tobogán revoleando su sombrilla de azafrán. Quien la pesque con una cañita de bambú, se la lleva a Siu Kiu.
Ya la luna viene en palanquin a robar un crisantemo del jardín Ya la luna viene por allí su kimono dice no, no y ella sí. Quien la pesque con una cañita de bambú, se la lleva a Siu Kiu.
Ya la luna baja muy feliz a empolvarse con azucar la nariz Ya la luna en puntas de pie en una tacita china toma té Quien la pesque con una cañita de bambú, se la lleva a Siu Kiu.
Ya la luna vino y le dio tos por comer con dos palitos el arroz Ya la luna baja desde allá y por el charquito-quito nadará Quien la pesque con una cañita de bambú, se la lleva a Siu Kiu


Vemos que logramos extraer la letra y filtrar lo que no deseábamos... Ahora, lo único que tenemos que hacer es encapsular todo en una función:

In [0]:
def getSong(song_url):
  
  links_ = requests.get(song_url)
  links_ = links_.content
  soup_ = BeautifulSoup(links_)

  title = soup_.find('h3').get_text()
  lyrics = []

  for i in soup_.find('div', {'class':'letra'}).find_all('p'): 
    if i.has_attr('class'):
      break
    elif i.a:
      pass
    else:
      lyrics.append((i.get_text(strip=True, separator=' ')))
    
  lyrics = ' '.join([str(elem) for elem in lyrics]) 

  song = {'title':title, 'lyrics':lyrics}
  return song

Entonces, esta función hace lo siguiente:

- toma como argumento un link
- parsea el html
- guarda el título de la canción en el objeto `title`
- extrae y guarda la letra de la canción en la lista `lyrics`
- arma un diccionario `songs` donde un elemento es el título y otro la letra

Lo último que tenemos que hacer, entonces, es iterar sobre la lista de `links` que habíamos construido con la función `get_links()` más arriba:

In [0]:
songs = []

for l in links:
  print(l)
  songs.append(getSong(l))

https://www.musica.com/letras.asp?letra=1852580
https://www.musica.com/letras.asp?letra=1980215
https://www.musica.com/letras.asp?letra=1814885
https://www.musica.com/letras.asp?letra=1814888
https://www.musica.com/letras.asp?letra=1814874
https://www.musica.com/letras.asp?letra=1814872
https://www.musica.com/letras.asp?letra=1925118
https://www.musica.com/letras.asp?letra=1818721
https://www.musica.com/letras.asp?letra=1875775
https://www.musica.com/letras.asp?letra=2022998
https://www.musica.com/letras.asp?letra=1814882
https://www.musica.com/letras.asp?letra=1857711
https://www.musica.com/letras.asp?letra=1875774
https://www.musica.com/letras.asp?letra=2138077
https://www.musica.com/letras.asp?letra=1814883
https://www.musica.com/letras.asp?letra=1857705
https://www.musica.com/letras.asp?letra=1814889
https://www.musica.com/letras.asp?letra=1814873
https://www.musica.com/letras.asp?letra=1814887
https://www.musica.com/letras.asp?letra=2301961
https://www.musica.com/letras.asp?letra=

Y lo acomodamos en un dataframe...

In [0]:
df_songs_mewalsh = pd.DataFrame(songs)
df_songs_mewalsh

Solamente para emprolijar un poco el dataset, vamos a limpiar un poco el título:

In [0]:
df_songs_mewalsh['title'] = df_songs_mewalsh['title'].str.replace('\(Letra/Lyrics\)', '')

### PRÁCTICA INDEPENDIENTE
---
Intenten hacer un scraping de letras de otra artista para niñes: Panam.


In [0]:
# EMPEZAR A ESCRIBIR EL CODIGO ACA

## Obtené los links de musica.com que contiene las letras de Panam

## Parsea cada link y extraé letra y título


## Consultando APIs: spotify
---
Acabamos de hacer un scraping de las letras de bastantes canciones de María Elena Walsh.

Ahora bien, ¿podríamos hacer algún tipo de análisis sobre la música de tales canciones? Un enfoque posible sería tratar de descargar los audios de los temas y tratar de procesarlos. Pero podemos intentar otra cosa: usar la API de Spotify.

En efecto, [Spotify for developers](https://developer.spotify.com/) tiene una gran cantidad de APIs y consultas sumamente interesantes para descargar información. En este tutorial vamos a utilizar solamente las consultas de "Audio features".

Pueden encontrar la documentación de esta API [acá](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/). Las principales métricas son las siguientes: 

- __Danceability:__ Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.
- __Acousticness:__ A measure from 0.0 to 1.0 of whether the track is acoustic.
- __Energy:__ Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy.
- __Instrumentalness:__ Predicts whether a track contains no vocals. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content.
- __Liveness:__ Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live.
- __Loudness:__ The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track. Values typical range between -60 and 0 db.
- __Speechiness:__ Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value.
- __Tempo:__ The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.
- __Valence:__ A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).


Si hiciéramos las cosas de forma "cruda", digamos, podríamos intentar algo como lo siguiente:

```
curl -X GET "https://api.spotify.com/v1/audio-features/06AKEBrKUckW0KREUWRnvT"
 -H "Authorization: Bearer {your access token}"
```

donde `06AKEBrKUckW0KREUWRnvT` es el ID del track. Esto nos devolvería una salida similar a esta:

```
{
  "duration_ms" : 255349,
  "key" : 5,
  "mode" : 0,
  "time_signature" : 4,
  "acousticness" : 0.514,
  "danceability" : 0.735,
  "energy" : 0.578,
  "instrumentalness" : 0.0902,
  "liveness" : 0.159,
  "loudness" : -11.840,
  "speechiness" : 0.0461,
  "valence" : 0.624,
  "tempo" : 98.002,
  "id" : "06AKEBrKUckW0KREUWRnvT",
  "uri" : "spotify:track:06AKEBrKUckW0KREUWRnvT",
  "track_href" : "https://api.spotify.com/v1/tracks/06AKEBrKUckW0KREUWRnvT",
  "analysis_url" : "https://api.spotify.com/v1/audio-analysis/06AKEBrKUckW0KREUWRnvT",
  "type" : "audio_features"
}

```
El chiste en este caso es lo que está entre comillas: el `access token`.  En este caso, y para simplifcar un poco las cosas, en lugar de hacer consultas directas a la API y escribir todas las requests, vamos a utilizar un paquete de Python que es "wrapper" de estas consultas. 

Primero, instalamos la librería que nos va a ayudar a consultar la API de Spotify: [spotipy](https://spotipy.readthedocs.io/en/2.11.1/).

In [0]:
!pip install spotipy

### PRACTICA INDEPENDIENTE. Crear una cuenta en Spotify Developers
---

Lo primero que hay que hacer es ir a [Spotify for Developers](https://developer.spotify.com/) y crear una cuenta (si ya tienen una cuenta en Spotify, pueden ingresar con ella).

Luego, ingresar a Dashboard y crear una App nueva. Llenan el formulario y obtendrán un id de cleinte y una secret key. Tomen nota de esos datos porque los vamos a necesitar para lo que sigue.


### Credenciales
---
Como vimos antes, el manejo de credenciales en las API puede ser un tanto engorroso. Lo bueno de usar `spotipy` es que nos permite trabajar todas las autentificaciones de forma simple. Lo único que tenemos que hacer es definir dos strings: 

- `cid`: nuestro ID
- `secret`: nuestro password secreto  

In [0]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="c58fff53289649e49247e7e3115f378c" 
secret = "fb89bbaa12b0438daf04fc46ac9f1ac6"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Creamos, entonces, dos objetos:

- `client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)` que va a manejar la autentificación
- `sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)` que va a ser el objeto con el que vamos a hacer las consultas


### Buscando a Manuelita... digo, a Maria Elena
---
Lo primero que vamos a necesitar es obtener lo que Spotify llama "ids" para cada uno de los temas de los que queremos extraer los audio features.

Para eso vamos a hacer lo siguiente:

In [0]:
track_results = sp.search(q='artist:'+'maria elena walsh', type='track', limit=50)

Aqui acabamos de hacer una consulta: buscamos, usando el parámetro `artist` a maria elena walsh. Esto nos devuelve un diccionario un tanto complejo. Veamos el primer elemento.

Para eso, tenemos que entrar al key `tracks` y al key `items`del diccionario. Eso nos devuelve una lista de diccionarios. Veamos el primer elemento:

In [0]:
track_results['tracks']['items'][0]

Vemos que hay muuucha información. De toda la información de este diccionario nos vamos a quedar con el nombre del artista, del track, la popularidad y el id (que es lo que vamos a necesitar para traer los features de audio. 

Entonces, primero vamos a crear listas vacías para cada una de estas variables:

In [0]:
artist_name = []
track_name = []
popularity = []
track_id = []

Y luego vamos a iterar por todo el objeto:

In [0]:
for i, t in enumerate(track_results['tracks']['items']):
  artist_name.append(t['artists'][0]['name']) # guardamos el nombre del artista
  track_name.append(t['name']) # el nombre del track 
  track_id.append(t['id']) # el id
  popularity.append(t['popularity']) # la popularidad

Y guardamos el resultado en un dataframe.

In [0]:
song_info_mewalsh = pd.DataFrame({'id':track_id, 'track_name': track_name, 'artist_name': artist_name, 'popularity': popularity})

### Trayendo audio features
---
Todo el código anterior fue para poder hacernos de los ids de las canciones que necesitamos para poder hacer una consulta y traer los audio features.

Entonces, vamos a iterar sobre los ids (que estaban guardados en la lista `track_id` y vamos a usar la consulta sobre `audio_features`:

In [0]:
audio_feat = []
for ident in track_id:
  a = sp.audio_features(ident)[0]
  audio_feat.append(a)

Y, una vez más, lo pasamos a un dataframe:

In [0]:
audio_feat_mewalsh = pd.DataFrame(audio_feat)

Por último, integramos todo en un solo data frame:

In [0]:
song_info_mewalsh = song_info_mewalsh.join(audio_feat_mewalsh.set_index('id'), on='id')

In [0]:
song_info_mewalsh

In [0]:
def dump(obj, file):
  import pickle
  from google.colab import files

  output = open(file, 'wb')
  pickle.dump(obj, output)
  files.download(file)

#dump(track_results, 'track_results_me.pkl')
# dump(audio_feat, 'audio_feat_me.pkl')

## PRACTICA INDEPENDIENTE
La idea es que ahora puedan buscar ustedes los audio features de Panam. Para ello, les dejamos esta pistas:

Hagan la búsqueda como "panam y circo"...


In [0]:
# EMPEZAR A ESCRIBIR EL CODIGO ACA

## Hacer la búsqueda en la API de tracks de panam y circo

## Extraer los IDS de los tracks

## Consultar la API y traer los audio features

## Transformar todo en un dataframe

Entonces, a partir de acá, podriamos empezar a comparar diferentes métricas entre María Elena Walsh y Panam...

In [0]:
## Comparar promedios de los features para PANAM y ME Walsh

## BONUS TRACK
Si se fijan en las funciones de sp.search() hay un límite de 50 resultados por request... Es que la corporación malvada de Spotify solamente nos muestra 50 resutlados por request.

Hay dos formas de solucionar esto... Una (simple) la pueden encontrar acá

https://github.com/tgel0/spotify-data/blob/master/notebooks/SpotifyDataRetrieval.ipynb

Básicamente, es anidar otro for loop y usar como parámetro en el offset.

Acá les mostramos otra... un poco más compleja:

El ID de Panam en Spotify es el siguiente.
`id_panam = '0hjh0CtlHZMyBcu4e8y8xW'`.

Entonces, pueden usar la función de búsqueda de álbumes en `spotipy` para obtener los ids de las canciones:


In [0]:
id_panam = '0hjh0CtlHZMyBcu4e8y8xW'
albums_panam = sp.artist_albums(id_panam)['items']

albums_panam_id = []
for i, t in enumerate(albums_panam):
  albums_panam_id.append(t['id'])

Luego, pueden iterar sobre los ids de los álbumes para obtener los ids de las  canciones y la información...

In [0]:
artist_name = []
track_name = []
popularity = []
track_id = []

for tr in albums_panam_id:
  album = sp.album_tracks(tr)['items']
  for i, t in enumerate(album):
    artist_name.append(t['artists'][0]['name']) # guardamos el nombre del artista
    track_name.append(t['name']) # el nombre del track 
    track_id.append(t['id']) # el id

Y, finalmente, traer los audio features...

In [0]:
audio_feat_panam = []
for ident in track_id:
  a = sp.audio_features(ident)[0]
  audio_feat_panam.append(a)

Si se fijan, ahora no tenemos 50 audio features sino 174...

In [0]:
len(audio_feat_panam)

174

## Solo en caso de emergencia...

In [0]:
import pickle

with open("/content/drive/My Drive/UBA Exactas/Clase web scraping/files/audio_feat_me.pkl", 'rb') as f:
  audio_feat_me =  pickle.load(f)

with open("/content/drive/My Drive/UBA Exactas/Clase web scraping/files/audio_feat_panam.pkl", 'rb') as f:
  audio_feat_panam =  pickle.load(f)

with open("/content/drive/My Drive/UBA Exactas/Clase web scraping/files/track_results_me.pkl", 'rb') as f:
  track_results_me =  pickle.load(f)

with open("/content/drive/My Drive/UBA Exactas/Clase web scraping/files/track_results_panam.pkl", 'rb') as f:
  track_results_panam =  pickle.load(f)

In [5]:
track_results_me

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=artist%3Amaria+elena+walsh&type=track&offset=0&limit=50',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5gMEZRCMq0gWA3kuCPukEk'},
       'href': 'https://api.spotify.com/v1/artists/5gMEZRCMq0gWA3kuCPukEk',
       'id': '5gMEZRCMq0gWA3kuCPukEk',
       'name': 'María Elena Walsh',
       'type': 'artist',
       'uri': 'spotify:artist:5gMEZRCMq0gWA3kuCPukEk'}],
     'available_markets': ['AD',
      'AE',
      'AR',
      'AT',
      'AU',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HU',
      'ID',
      'IE',
      'IL',
      'IN',
      'IS',
      'IT',
      '